In [1]:
import scipy.stats
import numpy as np

In [2]:
scipy.stats.norm.ppf(0.95)#95% Quantile,Probability of 95

1.6448536269514722

In [3]:
def var(Position,sigma,Quantile):#Sigma Daily Volatility 
    return Position*sigma*scipy.stats.norm.ppf(Quantile)

In [4]:
var(1000,0.05,0.95)#Vale at risk of the portfolio

82.24268134757361

In [5]:
#porfolio Variance=w_1^2*Sigma_1^2 + w_2^2*Sigma_2^2 +2*(w_1*w_2*sigma_1*sigma_2*corr) formula
w1=0.5#weight of the stock in portfolio list
w2=0.5#weight of the stock in portfolio list
sigma1=0.07#daily volatility
sigma2=0.03
corr=0.4#Correlation coefficient Those assets

In [6]:
porfolio_variance=w1**2*sigma1**2+w2**2*sigma2**2+2*(w1*w2*sigma1*sigma2*corr)

In [7]:
porfolio_volatility=porfolio_variance**(1/2)#square root

In [8]:
porfolio_volatility

0.043243496620879306

In [9]:
var(1000,porfolio_volatility,0.95)#1000 value of the portfolio and 95% confident

71.12922225891705

In [10]:
VaR1=var(1000*w1,sigma1,0.95)
VaR2=var(1000*w2,sigma2,0.95)

In [11]:
vector=np.array([VaR1,VaR2])
vector

array([57.56987694, 24.6728044 ])

In [12]:
corrmatrix=np.array([[1,corr],[corr,1]])
corrmatrix

array([[1. , 0.4],
       [0.4, 1. ]])

In [13]:
(np.dot(np.dot(vector,corrmatrix),vector))**(1/2)

71.12922225891707

In [14]:
import pandas_datareader.data as reader
import datetime as dt

In [15]:
end=dt.datetime.now()
start=dt.datetime(end.year-1,end.month,end.day)

In [16]:
df=reader.get_data_yahoo(['AAPL','MSFT','TSLA'],start,end)['Adj Close']
df

Symbols,AAPL,MSFT,TSLA
Date,,,
2021-08-18,145.544434,288.315369,688.989990
2021-08-19,145.882538,294.305176,673.469971
2021-08-20,147.364243,301.832092,680.260010
2021-08-23,148.875763,302.119690,706.299988
2021-08-24,148.786270,300.106598,708.489990
...,...,...,...
2022-08-12,172.100006,291.291718,900.090027
2022-08-15,173.190002,292.848389,927.960022
2022-08-16,173.029999,292.089996,919.690002


In [17]:
daily_returns=np.log(1+df.pct_change())
daily_returns

Symbols,AAPL,MSFT,TSLA
Date,,,
2021-08-18,NaN,NaN,NaN
2021-08-19,0.002320,0.020562,-0.022783
2021-08-20,0.010106,0.025254,0.010032
2021-08-23,0.010205,0.000952,0.037565
2021-08-24,-0.000601,-0.006686,0.003096
...,...,...,...
2022-08-12,0.021199,0.016894,0.045690
2022-08-15,0.006314,0.005330,0.030494
2022-08-16,-0.000924,-0.002593,-0.008952


In [18]:
log_return=np.log(df.AAPL/df.AAPL.shift(1)).dropna()
log_return

Date
2021-08-19    0.002320
2021-08-20    0.010106
2021-08-23    0.010205
2021-08-24   -0.000601
2021-08-25   -0.008457
                ...   
2022-08-12    0.021199
2022-08-15    0.006314
2022-08-16   -0.000924
2022-08-17    0.008746
2022-08-18   -0.002294
Name: AAPL, Length: 252, dtype: float64

In [19]:
daily_returns.std()

Symbols
AAPL    0.019163
MSFT    0.019077
TSLA    0.040026
dtype: float64

In [20]:
Position=df.iloc[-1]

In [21]:
Position

Symbols
AAPL    174.149994
MSFT    290.170013
TSLA    908.609985
Name: 2022-08-18 00:00:00, dtype: float64

In [22]:
VaRarray=[]
for i in range(len(Position)):
    VaRarray.append(var(Position[i],daily_returns.std()[i],0.95))

In [23]:
vector=np.array(VaRarray)
vector

array([ 5.48938116,  9.10498124, 59.82064776])

In [24]:
daily_returns.corr()

Symbols,AAPL,MSFT,TSLA
Symbols,,,
AAPL,1.000000,0.780028,0.644384
MSFT,0.780028,1.000000,0.578273
TSLA,0.644384,0.578273,1.000000


In [25]:
(np.dot(np.dot(vector,daily_returns.corr()),vector))**(1/2)#bcause Weight^2 volatility^1

69.44530888018497